# Отчет по заданию

Видео обзор этого задания и работы дашборда можно посмотреть по ссылке: https://youtu.be/qFHEQDPweeA

## Задание: 
<br/>

***Создать дашборд по звонкам в CallCenter***

<br/>
Пример визуализации Dashboard:
<br/>

![001.png](pics/001.png)
![002.png](pics/002.png)

<br/>

***Витрина данных:*** файл SLA_CONTACT_CENTRE.csv

***Описание витрины:***

№ | Наименование | Описание
:-|:-------------|:---------
1 | REPORT       | Тип телефонии с агрегацией. Например, Avaya By 15min – тип Avaya с разбивкой 15 минут
2 | DT           | Отчетная дата
3 | LD      | Дата загрузки в Хранилище данных
4 | DTTM | Дата и время периода агрегации
5 | SKILL_NM | Наименование скилла
6 | CALLS | Количество звонков
7 | ACCEPTED_CALLS | Количество принятых звонков
8 | RESET_CALLS | Количество непринятых звонков
9 | SLA | Нагрузка на операторов
10 | MAX_CALL_DT | Максимальное время звонка

***Параметры дашборда:***

1)	Отчетная дата (DT) – дата, на которую формируется отчет

***Блок Актуальность данных:***

1)	Телефония (Avaya и/или GENESYS)
2)	Максимальное время звонка (MAX_CALL_DT)
3)	Время загрузки данных по звонкам в Хранилище данных (LD)
4)	Отставание в минутах: разница от текущего времени и максимального времени звонка (пункт 2)

***Блок Отчетная дата*** – текущая дата формирования отчета из фильтра

***Блок Объем входящих звонков*** – общее количество звонков

***Блок SLA первый ответ в %*** (SLA)

***Блок количество непринятых звонков*** (RESET_CALLS)

***Блок SLA КЦ*** – график  с отображением объема звонков и нагрузки на операторов  за последние два дня (от отчетной даты).

***Блок Calls By Skills by Hour*** – объем звонков в разрезе скиллов по часам за текущие сутки (отчетная дата).

## Решение Задания:
<br/>
Создаю базу, таблицу, и наполню их данными:

``` mysql
CREATE DATABASE IF NOT EXISTS ccenter;

CREATE TABLE ccenter.sla_contact_center
(
REPORT String,
DT Date,
LD DateTime64,
DTTM DateTime64,
SKILL_NM String,
CALLS Int64,
ACCEPTED_CALLS Int64,
RESET_CALLS Int64,
SLA Float64,
MAX_CALL_DT Nullable(DateTime64)
) ENGINE = MergeTree ORDER BY (SKILL_NM) SETTINGS index_granularity = 8192;
```
<br/>
<br/>
Данные из CSV файла оказались немного разноформатными. Поэтому их надо причесать. Затем прямо отсюда, из ноутбука заливаю таблицу в Clickhouse. Для этого я использовал следующий код:

In [1]:
import pandas as pd
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.options.display.expand_frame_repr = False
pd.options.display.max_columns = None # display(df)
pd.options.display.max_rows = None
pd.options.display.min_rows = None

from clickhouse_driver import Client

df = pd.read_csv('./SLA_CONTACT_CENTRE.csv', sep=';')

df['DT'] = pd.to_datetime(df['DT'])
df['LD'] = pd.to_datetime(df['LD'])
df['DTTM'] = pd.to_datetime(df['DTTM'])
df['MAX_CALL_DT'] = df['MAX_CALL_DT'].fillna(0)
df['MAX_CALL_DT'] = pd.to_datetime(df['MAX_CALL_DT'])
df['REPORT'] = df['REPORT'].astype(str)
df['SKILL_NM'] = df['SKILL_NM'].astype(str)
df['SLA'] = pd.to_numeric(df['SLA'].str.replace("[,]", ".", regex=True),errors='coerce')
display(df.dtypes)
df.head(3)

/var/folders/xf/23jw26zs7r37__lj3ydbh14h0000gn/T/ipykernel_37325/4022281842.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


REPORT                    object
DT                datetime64[ns]
LD                datetime64[ns]
DTTM              datetime64[ns]
SKILL_NM                  object
CALLS                      int64
ACCEPTED_CALLS             int64
RESET_CALLS                int64
SLA                      float64
MAX_CALL_DT       datetime64[ns]
dtype: object

,REPORT,DT,LD,DTTM,SKILL_NM,CALLS,ACCEPTED_CALLS,RESET_CALLS,SLA,MAX_CALL_DT
0,Avaya By Day,2021-05-06,2021-09-09 17:43:00,2021-05-06 00:00:00,Itogo,52218,52008,210,0.962801,2021-05-06 23:58:51
1,Genesys By Day,2021-05-06,2021-08-05 23:59:00,2021-05-06 00:00:00,Itogo,367,367,0,1.000000,2021-05-06 08:18:21
2,Total By 15min,2021-05-06,2021-09-09 18:17:00,2021-05-06 08:30:00,Itogo,727,727,0,1.000000,2021-05-06 08:44:59


In [2]:
client = Client('10.8.10.119',
                database='ccenter',
                user='default'
                )
client.execute("INSERT INTO sla_contact_center VALUES", df.to_dict('records'))

2621

Данные загружены. <br/>
Теперь про Redash. Вы просили в письме показать как решаются задачи на этом инструменте. <br/>
Я сделал дашборд по присланному Вами образцу:<br/>
Вот его скрин:<br/>
    
![003.jpg](pics/003.jpg)
![004.jpg](pics/004.jpg)

Как это работает:<br/>
Дашборд состоит из белых плашек - прямоугольничков - виджетов. Каждый виджет - это SQL запрос в ClickHouse со своей визуализацией результирующей таблицы. <br/>
Вы прислали витрину с уже агрегированными данными. Поэтому SQL код виджетов простенький:<br/>

***Data actuality***

По этой таблице у меня масса вопросов. Хочется спросить заказчика как он будет мыслить, глядя на цифры из той или иной колонки. Какие выводы он будет делать, когда увидит там какие-то цифры. <br/>
У меня к Вам просьба: посмотрите код и скажите - правильно ли я понял задачу? И если будут уточнения, то сообщите мне, я переделаю этот виджет. 

``` mysql
select
    *
from
    (
    select
        CASE
        WHEN REPORT like '%Avaya%' THEN 'Avaya'
        WHEN REPORT like '%Genesys%' THEN 'Genesys'
        ELSE REPORT
        END as System,
        toDateTime(MAX_CALL_DT) as Call_time,
        toDateTime(LD) as Load_time,
        (now() - toDateTime(MAX_CALL_DT))/60 as Time_late_minutes
    from
        ccenter.sla_contact_center
    where
         DT = '{{ Date }}' and 
         (REPORT like '%Avaya%' or REPORT like '%Genesys%')
    
    UNION ALL
    
    select
        'Time_now' as System,
        now() as Call_time,
        now() as Load_time,
        0 as time_late_minutes
    )
order by System
```

Далее, В образце дашборда Объем звонков, SLA, и Количество потерянных выполнены в виде фактоидов без сегментации по АТС. Объем звонков и потерь окей это количественные показатели. Но вот SLA в витрине лежит в виде относительных величин в процентах. Поэтому объединять их неудачная идея. И я решил продолжить логику виджета "Data actuality" - сделать эти фактоиды с сегментацией по АТС:<br/>

***Volume_Avaya***

``` mysql
    select
        sum(CALLS)
    from
        ccenter.sla_contact_center
    where
         DT = '{{ Date }}' and 
         REPORT like '%Avaya%'
```

 ***Volume_Genesys***
 
``` mysql
    select
        sum(CALLS)
    from
        ccenter.sla_contact_center
    where
         DT = '{{ Date }}' and 
         REPORT like '%Genesys%'
```

 ***SLA_Avaya***
 
 ``` mysql
    select
        round(SLA*100, 2) as SLA
    from
        ccenter.sla_contact_center
    where
         DT = '{{ Date }}' and 
         REPORT like '%Avaya%'
```

 ***SLA_Genesys***
 
 ``` mysql
    select
        round(SLA*100, 2) as SLA
    from
        ccenter.sla_contact_center
    where
         DT = '{{ Date }}' and 
         REPORT like '%Genesys%'
```

 ***Abandoned_Avaya***
 
 ``` mysql
    select
        RESET_CALLS
    from
        ccenter.sla_contact_center
    where
         DT = '{{ Date }}' and 
         REPORT like '%Avaya%'
```

 ***Abandoned_Genesys***
 
 ``` mysql
    select
        RESET_CALLS
    from
        ccenter.sla_contact_center
    where
         DT = '{{ Date }}' and 
         REPORT like '%Genesys%'
```
 
Дальше идет график:<br/>

***SLA_CC_(15_minute)***

``` mysql
    select
        toDateTime(DTTM) as DTTM,
        CALLS,
        SLA*100 as SLA
    from
        ccenter.sla_contact_center
    where
         DT = '{{ Date }}' and 
         REPORT = 'Total By 15min'
    
```

И большая таблица:<br/>

***Volume_By_Skills_By_Hour***

``` mysql
select
    SKILL_NM,
    max(00_h) as 00_h,
    max(01_h) as 01_h,
    max(02_h) as 02_h,
    max(03_h) as 03_h,
    max(04_h) as 04_h,
    max(05_h) as 05_h,
    max(06_h) as 06_h,
    max(07_h) as 07_h,
    max(08_h) as 08_h,
    max(09_h) as 09_h,
    max(10_h) as 10_h,
    max(11_h) as 11_h,
    max(12_h) as 12_h,
    max(13_h) as 13_h,
    max(14_h) as 14_h,
    max(15_h) as 15_h,
    max(16_h) as 16_h,
    max(17_h) as 17_h,
    max(18_h) as 18_h,
    max(19_h) as 19_h,
    max(20_h) as 20_h,
    max(21_h) as 21_h,
    max(22_h) as 22_h,
    max(23_h) as 23_h
from
    (
    select
        SKILL_NM,
        CASE WHEN toHour(DTTM) = 0 THEN CALLS ELSE 0 END as 00_h,
        CASE WHEN toHour(DTTM) = 1 THEN CALLS ELSE 0 END as 01_h,
        CASE WHEN toHour(DTTM) = 2 THEN CALLS ELSE 0 END as 02_h,
        CASE WHEN toHour(DTTM) = 3 THEN CALLS ELSE 0 END as 03_h,
        CASE WHEN toHour(DTTM) = 4 THEN CALLS ELSE 0 END as 04_h,
        CASE WHEN toHour(DTTM) = 5 THEN CALLS ELSE 0 END as 05_h,
        CASE WHEN toHour(DTTM) = 6 THEN CALLS ELSE 0 END as 06_h,
        CASE WHEN toHour(DTTM) = 7 THEN CALLS ELSE 0 END as 07_h,
        CASE WHEN toHour(DTTM) = 8 THEN CALLS ELSE 0 END as 08_h,
        CASE WHEN toHour(DTTM) = 9 THEN CALLS ELSE 0 END as 09_h,
        CASE WHEN toHour(DTTM) = 10 THEN CALLS ELSE 0 END as 10_h,
        CASE WHEN toHour(DTTM) = 11 THEN CALLS ELSE 0 END as 11_h,
        CASE WHEN toHour(DTTM) = 12 THEN CALLS ELSE 0 END as 12_h,
        CASE WHEN toHour(DTTM) = 13 THEN CALLS ELSE 0 END as 13_h,
        CASE WHEN toHour(DTTM) = 14 THEN CALLS ELSE 0 END as 14_h,
        CASE WHEN toHour(DTTM) = 15 THEN CALLS ELSE 0 END as 15_h,
        CASE WHEN toHour(DTTM) = 16 THEN CALLS ELSE 0 END as 16_h,
        CASE WHEN toHour(DTTM) = 17 THEN CALLS ELSE 0 END as 17_h,
        CASE WHEN toHour(DTTM) = 18 THEN CALLS ELSE 0 END as 18_h,
        CASE WHEN toHour(DTTM) = 19 THEN CALLS ELSE 0 END as 19_h,
        CASE WHEN toHour(DTTM) = 20 THEN CALLS ELSE 0 END as 20_h,
        CASE WHEN toHour(DTTM) = 21 THEN CALLS ELSE 0 END as 21_h,
        CASE WHEN toHour(DTTM) = 22 THEN CALLS ELSE 0 END as 22_h,
        CASE WHEN toHour(DTTM) = 23 THEN CALLS ELSE 0 END as 23_h
    from
        ccenter.sla_contact_center
    where
         DT = '{{ Date }}' and 
         REPORT = 'Total Skills By Hour'
    )
group by SKILL_NM
order by SKILL_NM
```

Пока делал дашборд, меня не отпускала мысль о том, что дашборд этот "что-то скрывает". Очень уж он грубо агрегирован. За целый день. Для руководителей высшего звена, наверное, достаточно. Но вот для начальника смены (его еще называют трафик менеджером) захотелось сделать другой дашборд.<br/>
Какова суть работы трафик менеджера? Следить за метриками в режиме реального времени.<br/>
За какими метриками? По данным этой витрины есть возможность построить график SLA, Объема звонков, и потерь. Только само количество потерянных звонков плохо интерпретируется, поэтому есть метрика - доля потерянных звонков - Abandonment Rate. <br/>
И самое главное - как сегментировать эти метрики. Сегментация обычно проводится по скил группам. Потому что по метрикам есть таргеты (целевые значения)  И таргеты эти разные по разным тематикам запросов. Например, звонкам по блокировке потерянных карт нужно ставить строгие таргеты. Их нужно брать очень быстро. Почему? Потому что каждую секунду ожидания клиент думает о своих деньгах, которыми может завладеть кто-то чужой. И время для клиента длится секунда за час. Если клиент будет ждать слишком долго, то он потеряет лояльность к компании.<br/>
Звонки для контрагентов в бухгалтерию могут иметь менее строгие таргеты. <br/>
И так далее. <br/>
И получается, что метрики нужно контролировать в разрезе скиллов.<br/>
Дальше. <br/>
Про агрегацию: <br/>
Если строить отчет с квантованием за день - то усреднение будет скрывать проблемы. Случай из практики: В одном КЦ операторы любили опаздывать утром. Кто-то приходил на полчаса позже. кто-то на час. И, закономерно, в отзывах о компании среди клиентов ходила такая слава: "С утра им лучше не звонить - музыка играет, а трубку никто не берет".<br/>
Почему начальник ничего не замечал? Потому что он смотрел отчеты с квантованием по дням,по неделям, и по месяцам. А эти отчеты были красивые. потому что операторы роняли SLA в первый час каждого рабочего дня. Зато потом они деражали SLA под 100% весь день. И отчет за день сглаживался. <br/>
Поэтому я буду квантовать данные по 15 минут. (Обычно хватает и 30-ти минут)<br/>
<br/>
Вот такой у меня получился дашборд:<br/>

![005.jpg](pics/005.jpg)
![006.jpg](pics/006.jpg)

SQL код дашборда "Call_Center_V2":<br/>

***Skills_Distribution***

``` mysql

    select
        SKILL_NM,
        sum(CALLS) as Volume,
        sum(RESET_CALLS) as Abandoned,
        max(SLA)*100 as SLA_MAX,
        min(SLA)*100 as SLA_MIN,
        sum(RESET_CALLS)/sum(CALLS)*100 as Abandonment_Rate
    from
        ccenter.sla_contact_center
    where
         DT = '{{ Date }}' and 
         REPORT = 'Total Skills By 15min'
    group by SKILL_NM
    order by SKILL_NM
```

***SLA_ CC_chdp_(15_minute)***

``` mysql
    select
        toDateTime(DTTM) as DTTM,
        CALLS,
        SLA*100 as SLA,
        RESET_CALLS/CALLS*100 as Abandonment_Rate
    from
        ccenter.sla_contact_center
    where
         DT = '{{ Date }}' and 
         REPORT = 'Total Skills By 15min' and
         SKILL_NM = 'CC_chdp'
    
    
```

***SLA_ ConciergeService_(15_minute)***

``` mysql
    select
        toDateTime(DTTM) as DTTM,
        CALLS,
        SLA*100 as SLA,
        RESET_CALLS/CALLS*100 as Abandonment_Rate
    from
        ccenter.sla_contact_center
    where
         DT = '{{ Date }}' and 
         REPORT = 'Total Skills By 15min' and
         SKILL_NM = 'ConciergeService'
    
    
```

***SLA_ ContactCenter_(15_minute)***

``` mysql
    select
        toDateTime(DTTM) as DTTM,
        CALLS,
        SLA*100 as SLA,
        RESET_CALLS/CALLS*100 as Abandonment_Rate
    from
        ccenter.sla_contact_center
    where
         DT = '{{ Date }}' and 
         REPORT = 'Total Skills By 15min' and
         SKILL_NM = 'ContactCenter'
    
    
```

***SLA_ Customer_Service_(15_minute)***

``` mysql
    select
        toDateTime(DTTM) as DTTM,
        CALLS,
        SLA*100 as SLA,
        RESET_CALLS/CALLS*100 as Abandonment_Rate
    from
        ccenter.sla_contact_center
    where
         DT = '{{ Date }}' and 
         REPORT = 'Total Skills By 15min' and
         SKILL_NM = 'Customer_Service'
    
    
```

***SLA_ Postamat_(15_minute)***

``` mysql
    select
        toDateTime(DTTM) as DTTM,
        CALLS,
        SLA*100 as SLA,
        RESET_CALLS/CALLS*100 as Abandonment_Rate
    from
        ccenter.sla_contact_center
    where
         DT = '{{ Date }}' and 
         REPORT = 'Total Skills By 15min' and
         SKILL_NM = 'Postamat'
    
    
```

***SLA_ Predator_(15_minute)***

``` mysql
    select
        toDateTime(DTTM) as DTTM,
        CALLS,
        SLA*100 as SLA,
        RESET_CALLS/CALLS*100 as Abandonment_Rate
    from
        ccenter.sla_contact_center
    where
         DT = '{{ Date }}' and 
         REPORT = 'Total Skills By 15min' and
         SKILL_NM = 'Predator'
    
    
```

***SLA_ Save_(15_minute)***

``` mysql
    select
        toDateTime(DTTM) as DTTM,
        CALLS,
        SLA*100 as SLA,
        RESET_CALLS/CALLS*100 as Abandonment_Rate
    from
        ccenter.sla_contact_center
    where
         DT = '{{ Date }}' and 
         REPORT = 'Total Skills By 15min' and
         SKILL_NM = 'Save'
    
    
```

***SLA_ Second_Line_64_(15_minute)***

``` mysql
    select
        toDateTime(DTTM) as DTTM,
        CALLS,
        SLA*100 as SLA,
        RESET_CALLS/CALLS*100 as Abandonment_Rate
    from
        ccenter.sla_contact_center
    where
         DT = '{{ Date }}' and 
         REPORT = 'Total Skills By 15min' and
         SKILL_NM = 'Second_Line_64'
    
    
```

***SLA_ ContactCenter_(15_minute)***

``` mysql
    select
        toDateTime(DTTM) as DTTM,
        CALLS,
        SLA*100 as SLA,
        RESET_CALLS/CALLS*100 as Abandonment_Rate
    from
        ccenter.sla_contact_center
    where
         DT = '{{ Date }}' and 
         REPORT = 'Total Skills By 15min' and
         SKILL_NM = 'ContactCenter'
    
    
```

***SLA_ VQ_OAK_CC_Help_(15_minute)***

``` mysql
    select
        toDateTime(DTTM) as DTTM,
        CALLS,
        SLA*100 as SLA,
        RESET_CALLS/CALLS*100 as Abandonment_Rate
    from
        ccenter.sla_contact_center
    where
         DT = '{{ Date }}' and 
         REPORT = 'Total Skills By 15min' and
         SKILL_NM = 'VQ_OAK_CC_Help'
    
    
```